# Windows MCP Server Patch Demo

Demonstration of winpatch enabling MCP servers on Windows.

This notebook shows both uvx and npx based MCP servers working together after applying the Windows compatibility patch.

**Configuration**: Set `USE_OLLAMA = True` to use a local Ollama instance, or `False` to use OpenAI's cloud API.

In [ ]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from agents.mcp import MCPServerStdio
from openai import AsyncOpenAI
from winpatch import winpatch_mcpserver_stdio

# Configuration: Toggle between OpenAI and Ollama
USE_OLLAMA = False # Set to True to use Ollama, False for OpenAI
OLLAMA_MODEL = "qwen3:latest"  # Ollama model to use
OLLAMA_BASE_URL = "http://localhost:11434/v1"  # Ollama API base URL

if USE_OLLAMA:
    client = AsyncOpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")
    MODEL = OpenAIChatCompletionsModel(model=OLLAMA_MODEL, openai_client=client)
else:
    MODEL = "gpt-4o-mini"

In [ ]:
load_dotenv(override=True)

In [ ]:
# Apply Windows compatibility patch - enables MCP servers (uvx/npx) to work on Windows
winpatch_mcpserver_stdio()

## Test 1: UVX-based MCP Server (Fetch)

Testing that uvx-based MCP servers work correctly.

In [ ]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    tools = await server.list_tools()
    print(f"Fetch server loaded with {len(tools)} tool(s)")

tools

## Test 2: NPX-based MCP Server (Filesystem)

Testing that npx-based MCP servers work with the patch wrapping them with `cmd /c`.

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
os.makedirs(sandbox_path, exist_ok=True)

fs_params = {
    "command": "npx", 
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]
}

async with MCPServerStdio(params=fs_params, client_session_timeout_seconds=60) as server:
    tools = await server.list_tools()
    print(f"Filesystem server loaded with {len(tools)} tool(s)")
tools


## Test 3: Agent with Both Servers

Create an agent that uses both uvx and npx MCP servers together.

The agent will fetch content from a website and save it to a file using both servers.

### Tracing Configuration

When using Ollama, we configure a custom trace processor to capture execution traces locally. Unlike OpenAI's platform which provides built-in trace visualization at platform.openai.com/traces, Ollama requires a local tracing solution.

This gives you full visibility into the agent's execution flow, tool calls, and LLM interactions when running locally with Ollama.

In [ ]:
from agents.tracing import set_trace_processors, add_trace_processor
from custom_tracing_processor import CustomTraceProcessor

if USE_OLLAMA:
    set_trace_processors([CustomTraceProcessor()])
else:
    add_trace_processor(CustomTraceProcessor())

### Web fetch and write to filesystem

**Note**: The fetch tool may include extra content in responses even with the `raw` parameter. While `gpt-4o-mini` handles this better by filtering unwanted content, this doesn't affect the core demonstration of Windows MCP server compatibility.

In [ ]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
fs_params = {
    "command": "npx", 
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]
}

instructions = """
You are a specialized agent for downloading web content and saving it to files.

You have access to two tools:
1. fetch - downloads web content and returns it in the 'text' field. Always use the 'raw' parameter set to true.
2. write_file - saves text content to a file using 'path' and 'content' parameters

Always execute tools one at a time. After fetch returns, extract the text value and use it in write_file.
"""

prompt = """
Download the webpage from https://example.com and save it to sandbox/demo_jpt.html

Execute step by step:
1. Use fetch to download from https://example.com
2. After fetch succeeds, use write_file to save the downloaded text to sandbox/demo_jpt.html
"""

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as fetch_server:
    async with MCPServerStdio(params=fs_params, client_session_timeout_seconds=60) as fs_server:

        agent = Agent(
            name="demo_agent",
            instructions=instructions,
            model=MODEL,
            mcp_servers=[fetch_server, fs_server]
        )
        
        with trace("windows_no_wsl"):
            result = await Runner.run(
                agent, 
                prompt
            )
        
        print(f"Agent completed task")
        print(f"Result: {result.final_output}")
        if not USE_OLLAMA:
            print(f"\nView trace at: https://platform.openai.com/traces")
